In [37]:
import os, sys
import json
import cv2
import numpy as np

sys.path.insert(0, "/paddle/lcode/gry/PaddleNLP/")
from paddlenlp.transformers import ErnieGramTokenizer
tokenizer = ErnieGramTokenizer.from_pretrained('ernie-gram-zh')

lang = "zh"
mode = "val"
label_mode = "test"

# mode = "train"
# label_mode = "train"

max_norm_size = 1000
chunk_size = 509
set_path = "./data/XFUN_v1.0_data/" + lang + "/"
label_path = set_path + lang + "." + mode + ".json"
img_set_path = set_path + mode + "/"
label_set_dict = {}

save_label_path = set_path + label_mode + ".txt"
save_label_bbox_path = set_path + label_mode + "_box.txt"
save_label_image_path = set_path + label_mode + "_image.txt"
save_label_map_path = set_path + "labels.txt"
foutl = open(save_label_path, "wb")
foutb = open(save_label_bbox_path, "wb")
fouti = open(save_label_image_path, "wb")
foutlm = open(save_label_map_path, "wb")

def get_norm_resize_bbox(box, img_height, img_width, max_norm_size=1000):
    im_size_max = max(img_height, img_width)
    im_scale = float(max_norm_size) / float(im_size_max)
    box = np.array(box) * im_scale
    box = [int(np.round(i)) for i in box]
    return box

with open(label_path, "rb") as fin:
    lines = fin.readlines()
    label_data = json.loads(lines[0])['documents']
    sample_num = len(label_data)
    for sno in range(sample_num):
        if sno % 10 == 0:
            print(sno)
        img_id = label_data[sno]['id']
        document = label_data[sno]['document']
        bbox_num = len(document)
        img_path = img_set_path + img_id + ".jpg"
        img = cv2.imread(img_path)
        img_height, img_width = img.shape[0:2]
        
        tokens = []
        token_boxes = []
        actual_bboxes = []
        label_ids = []
        segment_ids = []
        for bno in range(bbox_num):
            label = document[bno]['label']
            words = document[bno]['words']
            words_num = len(words)
            for wno in range(words_num):
                word_text = words[wno]['text']
                actual_bbox = words[wno]['box']
                bbox = get_norm_resize_bbox(actual_bbox, img_height, img_width, max_norm_size)
                word_tokens = tokenizer.tokenize(word_text)
                tokens.extend(word_tokens)
                token_boxes.extend([bbox] * len(word_tokens))
                actual_bboxes.extend([actual_bbox] * len(word_tokens))
                if label == "other":
                    token_label = ["O"] * len(word_tokens)
                else:
                    token_label = [f"I-{label.upper()}"] * len(word_tokens)
                    token_label[0] = f"B-{label.upper()}"
                tmp_seg_idx = [bno] * len(word_tokens)
                label_ids.extend(token_label)
                segment_ids.extend(tmp_seg_idx)
        
        for chunk_id, index in enumerate(range(0, len(tokens), chunk_size)):
            beg_no = chunk_id * chunk_size
            end_no = min((chunk_id + 1) * chunk_size, len(tokens))
            for tno in range(beg_no, end_no):
                text = tokens[tno].strip("\n")
                label = label_ids[tno].strip("\n")
                bbox = token_boxes[tno]
                actual_bbox = actual_bboxes[tno]
                segment_id = segment_ids[tno]
                otstr = "%s\t%s\t%d\t%d\n" % (text, label, sno, segment_id)
                foutl.write(otstr.encode('utf-8'))

                otstr = "%s\t%d %d %d %d\t%d %d\t%s\n" % (
                    text, actual_bbox[0], actual_bbox[1], 
                    actual_bbox[2], actual_bbox[3],
                    img_width, img_height, img_path)
                fouti.write(otstr.encode('utf-8'))

                otstr = "%s\t%d %d %d %d\n" % (
                    text, bbox[0], bbox[1], bbox[2], bbox[3])
                foutb.write(otstr.encode('utf-8'))
                label_set_dict[label] = 0
                
            otstr = "\n"
            foutl.write(otstr.encode('utf-8'))
            foutb.write(otstr.encode('utf-8'))
            fouti.write(otstr.encode('utf-8'))
            
for label in label_set_dict:
    otstr = "%s\n" % label
    foutlm.write(otstr.encode('utf-8'))
foutl.close()
foutb.close()
fouti.close()
foutlm.close()
print("ok")

[2021-09-01 05:28:52,744] [    INFO] - Already cached ./.paddlenlp/models/ernie-gram-zh/vocab.txt


0
10
20
30
40


Corrupt JPEG data: 18 extraneous bytes before marker 0xc4


50
60
70
80
90
100
110
120
130
140


Corrupt JPEG data: bad Huffman code


ok
